In [7]:
# 導入必要的套件
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
# 建立 Ollama 模型
model = OllamaLLM(model="llama3.2:1b")

In [8]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_info}"),
    ]
)

In [9]:
pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。"),
        ]
)

pros_branch_chain = (
    pros_template
    | model
    | StrOutputParser()
)

In [10]:
cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。"),
        ]
)

cons_branch_chain =(
    cons_template
    | model
    | StrOutputParser() 
    
)

In [11]:
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

In [12]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chain,
        cons = cons_branch_chain
    )
    | RunnableLambda(lambda x: combine_pros_cons(x['pros'],x['cons']))
)

In [13]:
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_info": restaurant_info})

print(result)


🍽️ 餐廳評論分析報告

✅ 優點分析：
這家義大利餐廳具有以下優點：

* 环境優雅：室內装潑、木質板材等材料充分展現了意大利文化風格， create a warm and inviting atmosphere.
* 服務人員親切：พนักงาน在 phục vụ客人的時候也很Friendly、關懷，讓 khách人感到很舒適。
* 食物品質不錯：選擇的菜單中有很多好吃的意大利料理，food quality is satisfactory。

然而，這家餐廳也有以下缺點：

* 價格偏高：這家餐厅的價格相對較高，可能不เหมาะสำหร大多數人。
* 等待時間長：等待時間也較長，可能會影響到客人的體 experiens感受。

❌ 缺點分析：
這家義大利餐廳的優點包括：

* 环境優雅，餐厅的設計和美化是非常受欢迎的
* 服务人員親切、Friendly、關懷，讓客人感到很舒適

然而，這家餐廳也有一些缺點，包括：

*價格偏高：這家餐廳的價格相對較高，可能不เหมาะสำหร大多數人
* 等待時間長：等待時間也較長，可能會影響到客人的體驗感受

另外，我們需要注意一下，這家餐厅在台北市中心 location 的 pricing not 低廉，因此不是特別適合大多數人。

📊 總結：這是一個平衡的餐廳評論分析

